In [1]:
# load env variables from .env file
from dotenv import load_dotenv
import os
load_dotenv()


True

In [2]:

chatgpt_wed_api_key = os.environ.get("chatgpt_wed_api_key")
chatgpt_wed_region = os.environ.get("chatgpt_wed_region")
chatgpt_wed_endpoint = os.environ.get("chatgpt_wed_endpoint")
chatgpt_wed_api_version = os.environ.get("chatgpt_wed_api_version")
chatgpt_wed_deployment = os.environ.get("chatgpt_wed_deployment")
search_wed_endpoint = os.environ.get("search_wed_endpoint")
search_wed_key = os.environ.get("search_wed_key")
search_wed_index = os.environ.get("search_wed_index")

In [3]:
from urllib.parse import urljoin, urlparse 
aoai_extension_endpoint = urljoin(chatgpt_wed_endpoint, f"/openai/deployments/{chatgpt_wed_deployment}/extensions/chat/completions?api-version={chatgpt_wed_api_version}")
# chatgpt_api_url = urljoin(chatgpt_wed_endpoint, f"/openai/deployments/{chatgot_wed_deployment}/completions?api-version={chatgpt_wed_api_version}")
chatgpt_api_url = urljoin(chatgpt_wed_endpoint, f"/openai/deployments/{chatgpt_wed_deployment}/chat/completions?api-version=2023-03-15-preview")


In [4]:
print(f"AOAI Endpoint: {aoai_extension_endpoint}")
print(f"ChatGPT Endpoint: {chatgpt_api_url}")

AOAI Endpoint: https://magoai.openai.azure.com/openai/deployments/chat16k/extensions/chat/completions?api-version=2023-06-01-preview
ChatGPT Endpoint: https://magoai.openai.azure.com/openai/deployments/chat16k/chat/completions?api-version=2023-03-15-preview


In [5]:
import json 

def fprint(obj):
    # cast obj to dict before printing



    print(json.dumps(dict(obj), indent=4))


messages = { 
    "messages": [
        {
            "role": "system",
		    "content": """Assistant helps the company employees with their healthcare plan questions, and questions about the employee handbook. 
Be brief in your answers.
Say goodbye at the end of every response. 
            """
        }, 
        {
            "role": "user", 
            "content": "Hi, I'm looking for a wedding venue in the UK. Can you help me?"
        }

    ]
}


##TODO:  Include feilds mapping here
datasources = {
    "dataSources": [
        {
            "type": "AzureCognitiveSearch",
            "parameters": {
                "endpoint": search_wed_endpoint,
                "key": search_wed_key,
                "indexName": search_wed_index,
                "fieldsMapping": {
                    "titleField": "sourcefile",
                    "urlField": "sourcepage",
                    "filepathField": "sourcepage",
                    "contentFields": [
                        "content"
                    ],
                    "contentFieldsSeparator": "\n"
                },
                "inScope": "true",
                "topNDocuments": 3,
                "queryType": "semantic",
                "semanticConfiguration": "default",
                "roleInformation": ""
            }
        }
    ]
}


headers = {
    "Content-Type": "application/json",
    "api-key": f"{chatgpt_wed_api_key}",
    "chatgpt_url": chatgpt_api_url, 
    "chatgpt_key": chatgpt_wed_api_key,
}



In [6]:
print(headers.keys())

dict_keys(['Content-Type', 'api-key', 'chatgpt_url', 'chatgpt_key'])


In [7]:

# combine datasources and messages into a payload
payload = {**datasources, **messages}
#fprint(payload)


In [8]:
import requests
response = requests.post(aoai_extension_endpoint, headers=headers, data=json.dumps({**datasources, **messages}))
fprint(response.json())


{
    "id": "e0763eb9-22b6-46b1-8435-8e6f4b83ed86",
    "model": "gpt-35-turbo-16k",
    "created": 1693885904,
    "object": "chat.completion",
    "choices": [
        {
            "index": 0,
            "messages": [
                {
                    "index": 0,
                    "role": "tool",
                    "content": "{\"citations\": [], \"intent\": \"[\\\"I'm looking for a wedding venue in the UK\\\"]\"}",
                    "end_turn": false
                },
                {
                    "index": 1,
                    "role": "assistant",
                    "content": "The requested information is not found in the retrieved data. Please try another query or topic.",
                    "end_turn": true
                }
            ]
        }
    ]
}


In [9]:
# read response as dict 
response_dict = response.json()

response_dict["choices"][0]['messages']

for m in response_dict["choices"][0]['messages']: 
    if m["role"] == "assistant":
        messages["messages"].append(m)
    

In [10]:
fprint(messages)


{
    "messages": [
        {
            "role": "system",
            "content": "Assistant helps the company employees with their healthcare plan questions, and questions about the employee handbook. \nBe brief in your answers.\nSay goodbye at the end of every response. \n            "
        },
        {
            "role": "user",
            "content": "Hi, I'm looking for a wedding venue in the UK. Can you help me?"
        },
        {
            "index": 1,
            "role": "assistant",
            "content": "The requested information is not found in the retrieved data. Please try another query or topic.",
            "end_turn": true
        }
    ]
}


In [11]:
messages["messages"].append({
    "role": "user",
    "content": "tell me about northwind health plan"
})

fprint(messages)

response = requests.post(aoai_extension_endpoint, headers=headers, data=json.dumps({**datasources, **messages}))


{
    "messages": [
        {
            "role": "system",
            "content": "Assistant helps the company employees with their healthcare plan questions, and questions about the employee handbook. \nBe brief in your answers.\nSay goodbye at the end of every response. \n            "
        },
        {
            "role": "user",
            "content": "Hi, I'm looking for a wedding venue in the UK. Can you help me?"
        },
        {
            "index": 1,
            "role": "assistant",
            "content": "The requested information is not found in the retrieved data. Please try another query or topic.",
            "end_turn": true
        },
        {
            "role": "user",
            "content": "tell me about northwind health plan"
        }
    ]
}


In [12]:
len(response.json()['choices'])

1

In [13]:
fprint(response.json()['choices'][0])

{
    "index": 0,
    "messages": [
        {
            "index": 0,
            "role": "tool",
            "content": "{\"citations\": [{\"content\": \"Contoso Electronics Plan and Benefit Packages\\nContoso Electronics This document contains information generated using a language model (Azure OpenAI). The information contained in this document is only for demonstration purposes and does not reflect the opinions or beliefs of Microsoft. Microsoft makes no representations or warranties of any kind, express or implied, about the completeness, accuracy, reliability, suitability or availability with respect to the information contained in this document.\\nAll rights reserved to Microsoft Welcome to Contoso Electronics! We are excited to offer our employees two comprehensive health insurance plans through Northwind Health.\\nNorthwind Health Plus\\nNorthwind Health Plus is a comprehensive plan that provides comprehensive coverage for medical, vision, and dental services. This plan also o

In [14]:
# read citations from response 
citations = []
intent = []
for m in response.json()['choices'][0]['messages']: 
    if m["role"] == "tool":
        citations = json.loads(m["content"])['citations']
        intent = json.loads(m["content"])['intent']

    if m["role"] == "assistant":
        messages["messages"].append(m)

print(len(citations))

3


In [15]:
for c in citations:
    fprint(c)

{
    "content": "Contoso Electronics Plan and Benefit Packages\nContoso Electronics This document contains information generated using a language model (Azure OpenAI). The information contained in this document is only for demonstration purposes and does not reflect the opinions or beliefs of Microsoft. Microsoft makes no representations or warranties of any kind, express or implied, about the completeness, accuracy, reliability, suitability or availability with respect to the information contained in this document.\nAll rights reserved to Microsoft Welcome to Contoso Electronics! We are excited to offer our employees two comprehensive health insurance plans through Northwind Health.\nNorthwind Health Plus\nNorthwind Health Plus is a comprehensive plan that provides comprehensive coverage for medical, vision, and dental services. This plan also offers prescription drug coverage, mental health and substance abuse coverage, and coverage for preventive care services. With Northwind Healt

In [16]:
print(intent)

["tell me about northwind health plan"]
